### Modélisation des données du Défi IA 2021-2022

Dans ce calepin, nous décrivons quelques éléments de modélisation des données du  Défi IA 2021-2022. Il est suggéré quelques pistes pour réaliser des statistiques descriptives de ces données ainsi qu'une approche de prévision du cumul de pluie à l'aide de la régression linéaire. Des codes en Python sont également proposés. Le but de calepin est également de vous aider à initier la rédaction d'un rapport sur votre travail dans ce Défi IA.

#### Données disponibles sur les stations de mesure

Sur les années 2016 et 2017, données d'apprentissage sur $N$ stations météorologiques dont on dispose des coordonnées spatiales (latitude et longitude).

Pour chaque station $1 \leq i \leq N$, on dispose des mesures suivantes :

**Variables explicatives** : mesure de $p$ variables $X_{ijt} = (X_{ijt}^{(k)})_{1 \leq k \leq p} \in \mathbb{R}^{p}$ pour la station $i$, le jour $j$ (variable non-ordonnée car non-disponible dans l'ensemble test) et l'heure $t \in \{0,\ldots,23 \}$ (variable ordonnée disponible dans l'ensemble test). Les mesures sont

- 'ff' : *inclure une description*
- 't' : *inclure une description*
- 'td' : *inclure une description*
- 'hu' : *humidité*
- 'dd' : *inclure une description*
- 'precip' : *cumul de pluie sur une heure en ml*

On peut également ajouter une variable sur le mois de l'année car cette information est disponible dans l'ensemble test.


**Variable à expliquer/prédire** : cumul de pluie $Y_{ij}$ sur une journée au jour $j+1$ dans la station $i$ à partir des données disponibles au jour $j$. Dans l'ensemble d'apprentissage, on dipose en fait de la variable $Y_{ijt}$ cumul de pluie  sur une journée au jour $j+1$ dans la station $i$ et àl'heure $t$. De façon évidente on a que (avec $T=23$)
$$
Y_{ij} = \sum_{t = 0}^{T} Y_{ijt}
$$

**Travail préliminaire** : proposer une analyse descriptive de ces données : boxplot, histogramme uni-varié, ACP pour étude des corrélation entre variables explicatives, etc...

**Modèles linéaires possibles de prévision du cumul de pluie** : 

*Modèle global temps par temps*

$$
Y_{ijt} = \theta_{0}^{t} + \sum_{k = 1}^{p} \theta_{k}^{t}X_{ijt}^{(k)} + \varepsilon_{ijt}
$$

et prévision par $\hat{Y}_{ij} = \sum_{t = 0}^{T} \hat{Y}_{ijt} $ où $\hat{Y}_{ijt} = \hat{\theta}_{0}^{t} + \sum_{k = 1}^{p} \hat{\theta}_{k}^{t}X_{ijt}^{(k)}$

*Modèle par station et temps par temps*

$$
Y_{ijt} = \theta_{0,i}^{t} + \sum_{k = 1}^{p} \theta_{k,i}^{t}X_{ijt}^{(k)} + \varepsilon_{ijt}
$$

où les cofficients du modèle linéaire varient selon la station de mesure.

*Modèle global avec agrégation du temps*

$$
Y_{ij} = \theta_{0} + \sum_{t = 0}^{T}  \sum_{k = 1}^{p} \theta_{k}^{t}X_{ijt}^{(k)} + \varepsilon_{ij}
$$

et bien d'autres modèles sont possibles !

In [2]:
#from google.colab import drive
#drive.mount('/content/drive',force_remount=True)

In [3]:
#import os
#os.chdir('/content/drive/My Drive/Données Massives/')

In [4]:
import matplotlib.pyplot as plt
from IPython.display import display


In [5]:
import pandas as pd
import datetime
import seaborn as sns
import numpy as np

# Suppression des messages d'erreur liés à des besoins de mise à jour de syntaxe en Python
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

warnings.filterwarnings("ignore")

In [6]:
# Lecture des données de l'ensemble d'apprentissage 

path = 'defi-ia-2022/Train/Train/X_station_train.csv'
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)

# Read the ground station data
def read_gs_data(fname):
    gs_data = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    gs_data = gs_data.sort_values(by=["number_sta","date"])
    return gs_data

x = read_gs_data(path)
x['number_sta']=x['number_sta'].astype('category')

# Tri par station puis par datea
x = x.sort_values(['number_sta','date'])
x

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


In [7]:
# Ajout des variables jour et heure
Xtrain = x
split_Id = Xtrain['Id'].str.split(pat="_", expand = True)
split_Id = split_Id.rename(columns={0: "number_sta_2", 1: "day", 2: "hour"})
Xtrain['number_sta_2'] = split_Id['number_sta_2']
Xtrain['day'] = split_Id["day"]
Xtrain['hour'] = split_Id["hour"]
Xtrain = Xtrain.drop("number_sta_2",axis=1)
display(Xtrain) 

,number_sta,date,ff,t,td,hu,dd,precip,Id,day,hour
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,0,0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,0,1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,0,2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,0,3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,0,4
...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,729,19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,729,20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,729,21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,729,22


In [8]:
#Ajout de mois 

Xtrain['month'] = pd.DatetimeIndex(Xtrain['date']).month
display(Xtrain) 

,number_sta,date,ff,t,td,hu,dd,precip,Id,day,hour,month
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,0,0,1
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,0,1,1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,0,2,1
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,0,3,1
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,729,19,12
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,729,20,12
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,729,21,12
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,729,22,12


In [9]:
################################################
# Inclure une analyse desciptive des données ! #
################################################

Barplot des précipitations

In [10]:
# bins = [0,1,2,4,5,6,7,8,9,10,20,30]
# data_bar_precip = Xtrain.groupby(pd.cut(Xtrain['precip'], bins=bins)).count()
# data_bar_precip['precip'].plot.bar()

In [11]:
# Xtrain.boxplot()

In [12]:
# corr_Xtrain = Xtrain.corr(method='pearson')
# corr_Xtrain
# plt.matshow(corr_Xtrain)
# plt.show()

Matrice de corrélation de Xtrain
On retrouve une corrélation négative entre la température et l'humidité ce qui semble logique
On constate aussi une forte corrélation positive entre la température et td

In [13]:
# sns.heatmap(corr_Xtrain,annot=True)
# plt.show()

In [14]:
#On rajoute deux variables 
#precip_bool: 1: il pleut 0: il ne pleut pas
#precip _ategories: 5 categories 0, 0.2, 0.4 , 0.6, 0.8+


#Xtrain_2017['col2'] = pd.cut(df['col1'], bins=[0, 10, 50, float('Inf')], labels=['xxx', 'yyy', 'zzz'])

#Xtrain_2017



In [15]:
# Exemple d'implémentation du modèle linéaire par station et temps par temps mais avec
# des coefficients qui ne dépendent pas du temps
# Imputation de données manquantes par la médiane des variables

################################################################################################
# Attention, une erreur de mosélisation s'est glissée dans ce code, saurez-vous la retrouver ? #
################################################################################################

from sklearn.linear_model import LinearRegression

def regression_bystation (x,num_station):

    X = x[x['number_sta']==num_station]
    Y = X[{"date","precip"}]
    Y.set_index('date',inplace = True) 

    X = X[{"date","ff","t","td","hu","dd"}]
    X.set_index('date',inplace = True)

    # Imputation des valeurs manquantes
    median_t = X['t'].median()
    X['t'] = X['t'].fillna(median_t)

    median_ff = X['ff'].median()
    X['ff'] = X['ff'].fillna(median_ff)

    median_td = X['td'].median()
    X['td'] = X['td'].fillna(median_td)

    median_hu = X['hu'].median()
    X['hu'] = X['hu'].fillna(median_hu)

    median_dd = X['dd'].median()
    X['dd'] = X['dd'].fillna(median_dd)

    median_pre = Y['precip'].median()
    Y['precip'] = Y['precip'].fillna(median_pre)

    lr= LinearRegression(normalize=False)
    lr.fit(X, Y)

    return(lr)

ACP normée avec deux composantes principales


In [16]:
from sklearn.preprocessing import StandardScaler

# Xtrain = Xtrain.fillna(method='ffill')
# features = ['ff','t','td','hu','dd']

# X_acp = Xtrain.loc[:,features].values
# X_acp = StandardScaler().fit_transform(X_acp)


# Y_acp = Xtrain.loc[:,'precip'].values



In [17]:
from sklearn.decomposition import PCA

# pca = PCA(n_components = 5)

# principalComponents = pca.fit_transform(X_acp)

# principalDf = pd.DataFrame(data = principalComponents
#              , columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4','principal component 5'])

# finalDf = pd.concat([principalDf, Xtrain[['precip']]], axis = 1)



On récupère les valeurs propres de nos composantes principales

In [18]:
# pca.explained_variance_

Notre première composante principale explique 37% de l'inertie tandis que la seconde composante principale explique 25 % de l'inertie.

In [19]:
# pca.explained_variance_ratio_

Visualisation des valeurs propres de nos composantes principales. 

In [20]:

# sns.set_theme(style='darkgrid')
# graph_variance = sns.lineplot(x=['1','2','3','4','5'],y = pca.explained_variance_)
# graph_variance.axhline(1,color= 'red')

On constate que les composantes principales 1 et 2 ont des valeurs propres supérieurs à 1. 
Tandis que la troisième composante principale à une valeur propre très proche de 1 c'est pourquoi on décide aussi de la garder pour la suite selon la règle de Kaiser.


In [21]:
# display(finalDf)

S'il pleut, la variable class prend comme valeur 1, sinon elle prend comme valeur 2.


In [22]:
# finalDf['class'] = np.where(finalDf['precip'] == 0,1,2)
# display(finalDf)



Scatter plot avec les deux premières composantes principales

In [23]:
# sns.scatterplot(data = finalDf,x= 'principal component 1', y = 'principal component 2', hue='class')

En 3 dimensions cela devient illisible.

In [24]:
# fig = plt.figure()
# fig.set_size_inches(15, 10.5)
# ax = fig.add_subplot(111, projection='3d')
# x = np.array(finalDf['principal component 1'])
# y = np.array(finalDf['principal component 2'])
# z = np.array(finalDf['principal component 3'])
# ax.scatter(x,y,z, marker="s", c=finalDf["class"], s=20, cmap="RdBu")
# ax.set_xlabel('Principal Component 1')
# ax.set_ylabel('Principal Component 2')
# ax.set_zlabel('Principal Component 3')
# ax.legend()
# plt.show()

On va tenter d'ajouter une variable explicative ayant une bonne corrélation avec la variable precip.Pour cela, on va utiliser l'algorithme knn qui va nous dire s'il pleut ou pas.
La nouvelle variable knn qui sera crée prendra en valeur la prédiction de la classe donnée par l'algorithme knn. ( soit il pleut soit il ne pleut pas)


In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
# knn = KNeighborsClassifier(n_neighbors=10)



In [26]:
Xtrain

,number_sta,date,ff,t,td,hu,dd,precip,Id,day,hour,month
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,0,0,1
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,0,1,1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,0,2,1
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,0,3,1
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,729,19,12
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,729,20,12
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,729,21,12
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,729,22,12


In [27]:
Xtrain = Xtrain.fillna(method='ffill')
Xtrain['class'] = np.where(Xtrain['precip'] == 0,1,2)


X = Xtrain.drop(['date','class','Id'],axis=1)
Y = Xtrain['class']
X['day'] = X['day'].astype('category')
X['hour'] = X['hour'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=4,stratify=Y)

In [28]:
# #Xtrain['year'] = Xtrain[]
# Xtrain = Xtrain.fillna(method='ffill')
# Xtrain_gp = Xtrain.groupby(by=['number_sta','day'])



On standardise nos données

In [29]:
#knn.fit(X_train,y_train)
#predictions = knn.predict(X_test)
#accuracy_score(y_test,predictions)

In [30]:
# knn.fit(X_train,y_train)

In [31]:

accuracy_score(y_test,np.repeat(1,y_test.shape[0]))


0.9045026507302831

En ne prédisant que des 1 (i.e jours où il pleut), on obtient une précision de 90%

In [32]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
# random_forest = RandomForestClassifier()
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3, random_state=4,stratify=Y)
# ss = StandardScaler()

# X_train = ss.fit_transform(X_train)
# X_test = ss.transform(X_test)

# random_forest.fit(X_train,y_train)


In [33]:
# predictions = random_forest.predict(X_test)
# accuracy_score(y_test,predictions)

In [34]:
# np.unique(predictions,return_counts=True)


In [35]:
# np.unique(y_test,return_counts=True)



In [36]:
# predictions = knn.predict(X_test)
# accuracy_score(y_test,predictions)

On obtient une précision de 91% avec knn (k=10)


On va maintenant tenter d'améliorer ce score en utilisant la méthode de validation croisée avec knn et en prenant une grille de valeur de k afin de choisir le k qui maximise la précision de notre modèle.


In [37]:
# from sklearn.model_selection import GridSearchCV 
# knn2 = KNeighborsClassifier() 
# param_grid = {"n_neighbors" : np.arange(1, 25)}
# knn_cv = GridSearchCV(knn2, param_grid, cv=3)
# knn_cv.fit(X, Y)

In [38]:
# from sklearn.model_selection import GridSearchCV 
# knn2 = KNeighborsClassifier() 
# param_grid = {"n_neighbors" : np.arange(1, 25)}

In [39]:
X = Xtrain
Y = X[{"date","precip"}]
Y.set_index('date',inplace = True) 

# Imputation des valeurs manquantes
#X = X[{"date","ff","t","td","hu","dd"}]
#X.set_index('date',inplace = True)

In [40]:
Xtrain

,number_sta,date,ff,t,td,hu,dd,precip,Id,day,hour,month,class
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,0,0,1,1
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,0,1,1,1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,0,2,1,1
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,0,3,1,1
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,0,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,729,19,12,1
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,729,20,12,1
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,729,21,12,1
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,729,22,12,1


In [41]:
# Réorganisation des données pour modèle avec aggrégation dans le temps
X["number_sta_day"] = Xtrain['number_sta'].astype(str) + '_' + Xtrain['day'].astype(str)
X

,number_sta,date,ff,t,td,hu,dd,precip,Id,day,hour,month,class,number_sta_day
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,0,0,1,1,14066001_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,0,1,1,1,14066001_0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,0,2,1,1,14066001_0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,0,3,1,1,14066001_0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,0,4,1,1,14066001_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,729,19,12,1,95690001_729
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,729,20,12,1,95690001_729
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,729,21,12,1,95690001_729
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,729,22,12,1,95690001_729


In [42]:
X.drop(['class','Id','number_sta'],axis=1,inplace= True)

In [43]:
X['ff_idx'] = 'ff_' + X["hour"].astype(str)
X['t_idx'] = 't_' + X["hour"].astype(str)
X['td_idx'] = 'td_' + X["hour"].astype(str)
X['hu_idx'] = 'hu_' + X["hour"].astype(str)
X['dd_idx'] = 'dd_' + X["hour"].astype(str)
X['precip_idx'] = 'precip_' + X["hour"].astype(str)

In [44]:
ff = X.pivot(index='number_sta_day',columns='ff_idx',values='ff')
t = X.pivot(index='number_sta_day',columns='t_idx',values='t')
td = X.pivot(index='number_sta_day',columns='td_idx',values='td')
hu = X.pivot(index='number_sta_day',columns='hu_idx',values='hu')
dd = X.pivot(index='number_sta_day',columns='dd_idx',values='dd')
precip = X.pivot(index='number_sta_day',columns='precip_idx',values='precip')

In [45]:
X_reshape = pd.concat([ff ,t,td,hu,dd,precip],axis=1)

In [46]:
X_reshape.reset_index(inplace=True)
X_reshape

,number_sta_day,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_21,precip_22,precip_23,precip_3,precip_4,precip_5,precip_6,precip_7,precip_8,precip_9
0,14066001_0,3.05,2.57,3.38,3.20,3.85,5.19,6.04,4.43,5.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14066001_1,4.73,4.22,9.45,11.51,11.52,10.79,10.53,9.66,10.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14066001_10,3.68,4.70,7.78,8.08,6.89,5.54,7.01,7.10,6.41,...,0.2,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14066001_100,2.09,2.75,6.80,6.40,6.73,5.62,4.92,5.49,5.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14066001_101,1.23,1.35,2.08,2.33,2.64,2.48,2.83,1.69,1.96,...,0.0,0.0,0.0,0.2,0.4,2.2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183742,95690001_725,8.48,9.21,7.40,7.22,6.82,6.88,7.09,5.77,6.43,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0
183743,95690001_726,7.76,9.47,7.49,10.45,9.44,12.49,12.29,12.05,11.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.8
183744,95690001_727,6.04,5.15,2.90,2.43,3.93,4.48,4.48,4.06,3.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183745,95690001_728,1.78,1.26,9.64,10.86,9.73,9.10,10.45,8.57,9.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Données de cumul de pluie
fname = 'defi-ia-2022/Train/Train/Y_train.csv'
param = 'Ground_truth'  #weather parameter name in the file ('Ground_truth' about Y and 'Prediction' about baseline)
data = pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)
data['number_sta'] = data['number_sta'].astype('category')
display(data)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0
...,...,...,...,...
183742,2017-12-31,86137003,5.0,86137003_729
183743,2017-12-31,86165005,3.2,86165005_729
183744,2017-12-31,86272002,1.8,86272002_729
183745,2017-12-31,91200002,1.6,91200002_729


In [48]:
Y = data[["Ground_truth","Id"]]
Y.rename(columns = {"Ground_truth":"Y","Id":"number_sta_day"},inplace=True)
Y.dropna(inplace=True)
display(Y)

,Y,number_sta_day
0,3.4,14066001_0
1,0.5,14126001_0
2,3.4,14137001_0
3,4.0,14216001_0
4,13.3,14296001_0
...,...,...
183742,5.0,86137003_729
183743,3.2,86165005_729
183744,1.8,86272002_729
183745,1.6,91200002_729


In [49]:
all_data = pd.merge(X_reshape,Y)

display(all_data)
all_data.dropna(inplace=True)

,number_sta_day,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_22,precip_23,precip_3,precip_4,precip_5,precip_6,precip_7,precip_8,precip_9,Y
0,14066001_0,3.05,2.57,3.38,3.20,3.85,5.19,6.04,4.43,5.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4
1,14066001_1,4.73,4.22,9.45,11.51,11.52,10.79,10.53,9.66,10.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.7
2,14066001_10,3.68,4.70,7.78,8.08,6.89,5.54,7.01,7.10,6.41,...,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,14066001_100,2.09,2.75,6.80,6.40,6.73,5.62,4.92,5.49,5.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.6
4,14066001_101,1.23,1.35,2.08,2.33,2.64,2.48,2.83,1.69,1.96,...,0.0,0.0,0.2,0.4,2.2,0.0,0.0,0.0,0.0,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162102,95690001_725,8.48,9.21,7.40,7.22,6.82,6.88,7.09,5.77,6.43,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,3.2
162103,95690001_726,7.76,9.47,7.49,10.45,9.44,12.49,12.29,12.05,11.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.8,0.0
162104,95690001_727,6.04,5.15,2.90,2.43,3.93,4.48,4.48,4.06,3.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.4
162105,95690001_728,1.78,1.26,9.64,10.86,9.73,9.10,10.45,8.57,9.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.4


In [50]:

Y = all_data['Y']
X = all_data.drop("Y",axis=1)
X

,number_sta_day,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_21,precip_22,precip_23,precip_3,precip_4,precip_5,precip_6,precip_7,precip_8,precip_9
0,14066001_0,3.05,2.57,3.38,3.20,3.85,5.19,6.04,4.43,5.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14066001_1,4.73,4.22,9.45,11.51,11.52,10.79,10.53,9.66,10.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14066001_10,3.68,4.70,7.78,8.08,6.89,5.54,7.01,7.10,6.41,...,0.2,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14066001_100,2.09,2.75,6.80,6.40,6.73,5.62,4.92,5.49,5.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14066001_101,1.23,1.35,2.08,2.33,2.64,2.48,2.83,1.69,1.96,...,0.0,0.0,0.0,0.2,0.4,2.2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162102,95690001_725,8.48,9.21,7.40,7.22,6.82,6.88,7.09,5.77,6.43,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0
162103,95690001_726,7.76,9.47,7.49,10.45,9.44,12.49,12.29,12.05,11.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.8
162104,95690001_727,6.04,5.15,2.90,2.43,3.93,4.48,4.48,4.06,3.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162105,95690001_728,1.78,1.26,9.64,10.86,9.73,9.10,10.45,8.57,9.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
X['day'] = X['number_sta_day'].str.rpartition("_")[2]
X['day'] = X['day'].astype('int')
X['day'] = X['day'] % 365

In [52]:
X['number_sta'] = X['number_sta_day'].str.rpartition('_')[0]
X['number_sta'] = X['number_sta'].astype('int64')

In [53]:
X['season'] =pd.cut(X['day'],[0,60,151,243,334], labels=['Hiver','Printemps','Ete', 'Automne'],ordered=False)
X['season'] = X['season'].fillna('Hiver')
X['season']

0             Hiver
1             Hiver
2             Hiver
3         Printemps
4         Printemps
            ...    
162102        Hiver
162103        Hiver
162104        Hiver
162105        Hiver
162106        Hiver
Name: season, Length: 162090, dtype: category
Categories (4, object): ['Hiver', 'Printemps', 'Ete', 'Automne']

In [54]:
X['month'] = X['day'] / 30
X['month'] = X['month'].astype('int')


In [55]:
X.reset_index(inplace=True)
X.drop('index',axis=1,inplace=True)

In [56]:
X['month'] = X['month'].astype('category')
X['season'] = X['season'].astype('category')
X['day'] = X['day'].astype('category')

Ajoutons les coordonnées des stations

In [57]:
#open the file with station coordinates (latitude/longitude)

coords_fname  = 'defi-ia-2022/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname)
coords['number_sta'] = coords['number_sta'].astype('category')
display(coords)



,number_sta,lat,lon,height_sta
0,86118001,46.477,0.985,120.0
1,86149001,46.917,0.025,60.0
2,56081003,48.050,-3.660,165.0
3,53215001,47.790,-0.710,63.0
4,22135001,48.550,-3.380,148.0
...,...,...,...,...
320,86137003,47.035,0.098,96.0
321,86165005,46.412,0.841,153.0
322,86273001,46.464,1.042,121.0
323,91200002,48.526,1.993,116.0


In [58]:
len(np.unique(X['number_sta']))

254

In [59]:
coords.dtypes

number_sta    category
lat            float64
lon            float64
height_sta     float64
dtype: object

In [60]:
X =pd.merge(X,coords,how='left',on='number_sta')
X['number_sta'] = X['number_sta'].astype('category')
X

,number_sta_day,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_7,precip_8,precip_9,day,number_sta,season,month,lat,lon,height_sta
0,14066001_0,3.05,2.57,3.38,3.20,3.85,5.19,6.04,4.43,5.10,...,0.0,0.0,0.0,0,14066001,Hiver,0,49.334,-0.431,2.0
1,14066001_1,4.73,4.22,9.45,11.51,11.52,10.79,10.53,9.66,10.26,...,0.0,0.0,0.0,1,14066001,Hiver,0,49.334,-0.431,2.0
2,14066001_10,3.68,4.70,7.78,8.08,6.89,5.54,7.01,7.10,6.41,...,0.0,0.0,0.0,10,14066001,Hiver,0,49.334,-0.431,2.0
3,14066001_100,2.09,2.75,6.80,6.40,6.73,5.62,4.92,5.49,5.41,...,0.0,0.0,0.0,100,14066001,Printemps,3,49.334,-0.431,2.0
4,14066001_101,1.23,1.35,2.08,2.33,2.64,2.48,2.83,1.69,1.96,...,0.0,0.0,0.0,101,14066001,Printemps,3,49.334,-0.431,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162085,95690001_725,8.48,9.21,7.40,7.22,6.82,6.88,7.09,5.77,6.43,...,0.0,0.0,0.0,360,95690001,Hiver,12,49.108,1.831,126.0
162086,95690001_726,7.76,9.47,7.49,10.45,9.44,12.49,12.29,12.05,11.24,...,0.0,1.4,0.8,361,95690001,Hiver,12,49.108,1.831,126.0
162087,95690001_727,6.04,5.15,2.90,2.43,3.93,4.48,4.48,4.06,3.16,...,0.0,0.0,0.0,362,95690001,Hiver,12,49.108,1.831,126.0
162088,95690001_728,1.78,1.26,9.64,10.86,9.73,9.10,10.45,8.57,9.81,...,0.0,0.0,0.0,363,95690001,Hiver,12,49.108,1.831,126.0


In [61]:
Y =pd.DataFrame(Y)
Y.reset_index(inplace=True)
Y.drop('index',inplace = True,axis=1)

In [62]:
Y

,Y
0,3.4
1,11.7
2,1.0
3,5.6
4,3.2
...,...
162085,3.2
162086,0.0
162087,4.4
162088,5.4


In [63]:
#X['class'] = np.where(Y['Y'] == 0,1,2)
#Y = X.loc[:,X.columns.str.startswith('precip')].sum(axis=1)
Class =np.where(Y['Y'] == 0,1,2)
Class = pd.DataFrame(Class)
Class.reset_index(inplace=True)
Class.drop('index',inplace=True,axis=1)

In [64]:
Class

,0
0,2
1,2
2,2
3,2
4,2
...,...
162085,2
162086,1
162087,2
162088,2


On souhaite enrichir notre base de données en utilisant des techniques de data augmentation tel que SMOTE.
En effet, comme notre base de données est très déséquilibré avec beaucoup plus de jours où il ne pleut pas, nous allons sur-représenter les jours où il pleut avec SMOTE afin d'améliorer notre classification. 

In [65]:
from sklearn.model_selection import ShuffleSplit,StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [74]:
from imblearn.over_sampling import SMOTE
transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )

X_smote = transformer.fit_transform(X)
sm = SMOTE(random_state=4)
X_smote,Class_smote = sm.fit_resample(X_smote,Class)


In [75]:
X_smote = pd.DataFrame(X_smote)
Class_smote = pd.DataFrame(Class_smote)
X_smote

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,0.000000,-0.585644,-0.727674,-0.751228,-0.863293,-0.670316,-0.214491,0.088655,-0.446376,-0.162822,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.200213,1.312095,0.012962,-1.578531
1,1.000000,-0.073116,-0.225456,1.306710,2.001254,2.001200,1.749053,1.664525,1.372796,1.595329,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.200213,1.312095,0.012962,-1.578531
2,2.000000,-0.393446,-0.079356,0.740523,0.818896,0.388538,-0.091770,0.429099,0.482341,0.283530,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.200213,1.312095,0.012962,-1.578531
3,3.000000,-0.878518,-0.672886,0.408270,0.239782,0.332809,-0.063719,-0.304435,-0.077672,-0.057197,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.200213,1.312095,0.012962,-1.578531
4,4.000000,-1.140884,-1.099010,-1.191973,-1.163191,-1.091768,-1.164706,-1.037969,-1.399442,-1.232705,...,0.268049,0.707930,4.912466,-0.201899,-0.196643,-0.197127,-0.200213,1.312095,0.012962,-1.578531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190045,148068.134890,0.805505,0.806373,0.682887,0.656882,0.639319,0.634040,0.639683,0.659737,0.692402,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.029350,-1.033399,-0.382222,-0.162215
190046,83744.741185,-1.245866,-1.070954,-0.922161,-0.890322,-0.824083,-0.903257,-0.986148,-1.063311,-1.054205,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.200213,-0.489688,0.262285,-0.277832
190047,47192.216164,-0.832519,-0.756478,-0.720157,-0.657571,-0.442017,-0.337851,-0.215810,-0.322731,-0.377374,...,-0.201406,-0.205068,-0.207578,-0.201899,-0.196643,-0.197127,-0.200213,0.463789,-0.382222,0.603752
190048,135781.863002,1.016008,1.016391,0.916821,0.894732,0.879651,0.875977,0.881855,0.899742,0.927504,...,-0.201406,-0.205068,-0.075272,-0.201899,-0.196643,-0.197127,-0.200213,-1.074078,-0.171909,0.560396


Utilisons KNN avec données issue de SMOTE

In [87]:

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X_smote, Class_smote)
train_index, test_index = next(split.split(X_smote, Class_smote)) 

X_train,X_test = X_smote.loc[train_index], X_smote.loc[test_index]
Y_train, Y_test = Class_smote.loc[train_index], Class_smote.loc[test_index]


In [88]:
transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)




In [89]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,Y_train)



predictions = knn.predict(X_test)
accuracy_score(Y_test,predictions)


0.7274576865737087

In [90]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]


Random Forest

In [91]:
random_forest = RandomForestClassifier(oob_score=True)

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

X_train
random_forest.fit(X_train,Y_train)


RandomForestClassifier(oob_score=True)

In [92]:
predictions = random_forest.predict(X_test)
accuracy_score(Y_test,predictions)

0.7422419643407984

On récupère les features importantes

In [93]:
# Get numerical feature importances
importances = list(random_forest.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(X.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: t_16                 Importance: 0.02
Variable: t_17                 Importance: 0.02
Variable: t_18                 Importance: 0.02
Variable: t_19                 Importance: 0.02
Variable: number_sta_day       Importance: 0.01
Variable: ff_20                Importance: 0.01
Variable: ff_21                Importance: 0.01
Variable: ff_22                Importance: 0.01
Variable: ff_23                Importance: 0.01
Variable: t_0                  Importance: 0.01
Variable: t_1                  Importance: 0.01
Variable: t_10                 Importance: 0.01
Variable: t_11                 Importance: 0.01
Variable: t_12                 Importance: 0.01
Variable: t_13                 Importance: 0.01
Variable: t_14                 Importance: 0.01
Variable: t_15                 Importance: 0.01
Variable: t_2                  Importance: 0.01
Variable: t_20                 Importance: 0.01
Variable: t_21                 Importance: 0.01
Variable: t_22                 Importanc

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Utilisons Support Vector Machine (SVM) afin de classifier les jours où il pleut

In [ ]:
# from sklearn import svm
# svm_model = svm.SVC(kernel = 'linear')
# svm_model.fit(X_train,Y_train)
# predictions = svm_model.predict(X_test)
# accuracy_score(Y_test,predictions)

In [ ]:
# predictions

Régression Logisitique

In [94]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)



In [95]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=4,class_weight = {1:0.7,2:1})
lg.fit(X_train,Y_train)


LogisticRegression(class_weight={1: 0.7, 2: 1}, random_state=4)

In [96]:
predictions = lg.predict(X_test)
accuracy_score(Y_test,predictions)

0.6405494889670348

XGBoost

In [97]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X, Class)
train_index, test_index = next(split.split(X, Class)) 

X_train,X_test = X.loc[train_index], X.loc[test_index]
Y_train, Y_test = Class.loc[train_index], Class.loc[test_index]

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

In [98]:
from xgboost import XGBClassifier
xg = XGBClassifier(n_estimators=500,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=-999,
    random_state=4,
    tree_method='gpu_hist')
xg.fit(X_train,Y_train)

[20:15:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              enable_categorical=False, gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=11, min_child_weight=1, missing=-999,
              monotone_constraints='()', n_estimators=500, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=4,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [99]:
predictions = xg.predict(X_test)
accuracy_score(Y_test,predictions)

0.7552388590700639

In [100]:
from xgboost import XGBClassifier
xg = XGBClassifier(n_estimators=2000,
    max_depth=6,
    learning_rate=0.02,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=4,
    objective = 'binary:logistic',
    nthread= -1,
    min_child_weight= 2,   
    tree_method='gpu_hist')
xg.fit(X_train,Y_train)

[20:16:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              enable_categorical=False, gamma=0, gpu_id=0, importance_type=None,
              interaction_constraints='', learning_rate=0.02, max_delta_step=0,
              max_depth=6, min_child_weight=2, missing=nan,
              monotone_constraints='()', n_estimators=2000, n_jobs=8,
              nthread=-1, num_parallel_tree=1, predictor='auto', random_state=4,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [101]:
predictions = xg.predict(X_test)
accuracy_score(Y_test,predictions)

0.7406790466201905

Extra Trees Classifier (possède une faible variance comparé à Random Forest)

In [144]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X_smote, Class_smote)
train_index, test_index = next(split.split(X_smote, Class_smote)) 

X_train,X_test = X_smote.loc[train_index], X_smote.loc[test_index]
Y_train, Y_test = Class_smote.loc[train_index], Class_smote.loc[test_index]

transformer = ColumnTransformer(
                     [
                         ('transform_name_categories', OrdinalEncoder(), make_column_selector(dtype_include=object)),
                         ('transformer_name_for_numerical', StandardScaler(), make_column_selector(dtype_include=np.number))
                     ]
                 )
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

In [145]:
from sklearn.ensemble import ExtraTreesClassifier
extree = ExtraTreesClassifier(
    random_state = 4,
    n_estimators =500 )
extree.fit(X_train,Y_train)

ExtraTreesClassifier(n_estimators=500, random_state=4)

In [146]:
predictions = extree.predict(X_test)
accuracy_score(Y_test,predictions)

0.7744628606507059

Nous allons utilisé un autre modèle pour prédire le cumul de pluie sur les jours où on a prédit qu'il pleuvait

Dans le cas où on n'utilise pas SMOTE

In [105]:
X_test = pd.DataFrame(X_test)
X_test['index_original'] = test_index
X_test

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,index_original
0,0.160435,0.497249,0.475090,0.227338,0.089899,0.071294,0.104209,-0.018549,-0.146969,-0.088247,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,1.100743,-0.670430,-1.510519,88974
1,-1.608070,2.360315,2.359237,2.418606,2.425251,2.427229,2.435160,2.443422,2.446666,2.441891,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,1.271531,-0.076707,-1.394550,6530
2,-1.577052,0.527841,0.530057,0.374105,0.343590,0.324092,0.316435,0.321522,0.344383,0.382716,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.982582,0.239224,-0.858192,7976
3,0.046788,-0.738677,-0.670055,-1.192549,-1.140331,-1.255894,-1.137314,-1.290272,-1.241981,-1.177994,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-0.478052,0.250040,-0.278346,83676
4,-0.750610,-0.952822,-0.786097,-1.370035,-1.366221,-1.420915,-1.370762,-1.191085,-0.947170,-0.875477,...,-0.201487,0.283999,-0.196694,-0.200996,-0.191175,-0.190492,0.325247,-0.664737,-0.350827,46503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57010,-0.973556,1.832097,1.861662,-0.185814,-0.526389,-0.099170,0.244377,0.697043,1.519642,1.648019,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.470218,-2.634326,-0.684239,166462
57011,1.085766,-1.417824,-1.204456,-1.059435,-0.792808,-0.971497,-0.765918,-0.780166,-0.796255,-0.961419,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,1.802761,0.567678,-0.365323,132111
57012,-0.755308,-1.185323,-0.923514,-1.062848,-0.893590,-0.883720,-0.914476,-1.095440,-1.340251,-1.109240,...,-0.201487,0.283999,1.836055,3.064335,1.450191,-0.190492,0.325247,-0.664737,-0.350827,46284
57013,-1.488974,0.059780,0.001764,-0.011585,-0.250673,-0.251725,0.514513,0.838713,0.270680,0.300212,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.750231,-1.730364,-0.379820,12082


In [106]:
index_pluie =np.where(predictions==2)

In [107]:
X_pluie = X_test.loc[index_pluie]
X_pluie

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,index_original
8,-0.656676,0.953073,0.954525,0.848538,0.826647,0.812132,0.808093,0.813916,0.832225,0.860555,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.531576,0.985504,0.214523,50882
10,0.449895,0.567611,0.569756,0.418476,0.388768,0.369736,0.362418,0.367573,0.390008,0.427406,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.066086,-1.056947,-0.539277,102468
13,1.002064,0.506426,0.508681,0.350213,0.319264,0.299514,0.291676,0.296725,0.319815,0.358653,...,-0.201487,0.283999,-0.196694,-0.200996,-0.191175,-0.190492,1.497924,1.221172,-0.104393,128209
14,-0.342119,-0.038115,-0.034881,-0.257335,-0.299326,-0.325458,-0.337929,-0.333823,-0.304903,-0.253256,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-0.657777,0.622895,-0.263850,65546
16,1.368040,0.200504,0.639991,-0.260748,-0.035209,-0.019994,0.051153,-0.355077,-1.217413,-1.425507,...,12.431747,3.891292,-0.196694,-0.200996,0.355947,-0.190492,-1.511717,-0.738170,-1.220596,145270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56996,1.135978,1.011198,1.012545,0.913389,0.892676,0.878843,0.875298,0.881222,0.898908,0.925871,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.702569,1.238249,0.939330,171231
57002,1.516952,0.283103,0.441500,0.039612,-0.288901,-0.332480,-0.256575,0.541151,0.088178,-0.490456,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.224753,-0.228696,0.605919,152212
57005,0.503633,1.317120,1.317918,1.254708,1.240199,1.229951,1.229008,1.235462,1.249873,1.269640,...,3.291514,2.243702,0.233102,0.259276,-0.191175,-0.190492,-0.082857,-1.659778,-0.524781,166991
57012,-0.755308,-1.185323,-0.923514,-1.062848,-0.893590,-0.883720,-0.914476,-1.095440,-1.340251,-1.109240,...,-0.201487,0.283999,1.836055,3.064335,1.450191,-0.190492,0.325247,-0.664737,-0.350827,46284


In [112]:
X_pluie.shape

(28900, 149)

In [113]:
Y_test = pd.DataFrame(Y_test)
Y_test

,0
88974,2
6530,2
7976,2
83676,1
46503,2
...,...
166462,1
132111,2
46284,2
12082,2


In [114]:
Y = pd.DataFrame(Y)
Y.reset_index(inplace=True)
Y.drop('index',axis=1,inplace=True)

In [116]:
Y

,Y
0,3.4
1,11.7
2,1.0
3,5.6
4,3.2
...,...
162085,3.2
162086,0.0
162087,4.4
162088,5.4


In [ ]:
Y_pluie = Y.loc[X_pluie['index_original']]



In [ ]:
Y_pluie = pd.DataFrame(Y_pluie)
Y_pluie = Y_pluie.reset_index()


In [ ]:
Y_pluie.drop('index',axis=1,inplace=True)

In [ ]:
Y_pluie

In [ ]:
sum(Y_pluie['Y']==0)

Dans le cas où on utilise SMOTE

In [147]:
X_test_smote = pd.DataFrame(X_test)
X_test_smote['index_original'] = test_index
X_test_smote

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,index_original
0,0.160435,0.497249,0.475090,0.227338,0.089899,0.071294,0.104209,-0.018549,-0.146969,-0.088247,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,1.100743,-0.670430,-1.510519,88974
1,-1.608070,2.360315,2.359237,2.418606,2.425251,2.427229,2.435160,2.443422,2.446666,2.441891,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,1.271531,-0.076707,-1.394550,6530
2,-1.577052,0.527841,0.530057,0.374105,0.343590,0.324092,0.316435,0.321522,0.344383,0.382716,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.982582,0.239224,-0.858192,7976
3,0.046788,-0.738677,-0.670055,-1.192549,-1.140331,-1.255894,-1.137314,-1.290272,-1.241981,-1.177994,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-0.478052,0.250040,-0.278346,83676
4,-0.750610,-0.952822,-0.786097,-1.370035,-1.366221,-1.420915,-1.370762,-1.191085,-0.947170,-0.875477,...,-0.201487,0.283999,-0.196694,-0.200996,-0.191175,-0.190492,0.325247,-0.664737,-0.350827,46503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57010,-0.973556,1.832097,1.861662,-0.185814,-0.526389,-0.099170,0.244377,0.697043,1.519642,1.648019,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.470218,-2.634326,-0.684239,166462
57011,1.085766,-1.417824,-1.204456,-1.059435,-0.792808,-0.971497,-0.765918,-0.780166,-0.796255,-0.961419,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,1.802761,0.567678,-0.365323,132111
57012,-0.755308,-1.185323,-0.923514,-1.062848,-0.893590,-0.883720,-0.914476,-1.095440,-1.340251,-1.109240,...,-0.201487,0.283999,1.836055,3.064335,1.450191,-0.190492,0.325247,-0.664737,-0.350827,46284
57013,-1.488974,0.059780,0.001764,-0.011585,-0.250673,-0.251725,0.514513,0.838713,0.270680,0.300212,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.750231,-1.730364,-0.379820,12082


In [148]:
index_pluie_smote =np.where(predictions==2)

In [156]:
X_pluie_smote = X_test_smote.loc[index_pluie_smote]
X_pluie_smote

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,index_original
8,-0.656676,0.953073,0.954525,0.848538,0.826647,0.812132,0.808093,0.813916,0.832225,0.860555,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.531576,0.985504,0.214523,50882
10,0.449895,0.567611,0.569756,0.418476,0.388768,0.369736,0.362418,0.367573,0.390008,0.427406,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.066086,-1.056947,-0.539277,102468
13,1.002064,0.506426,0.508681,0.350213,0.319264,0.299514,0.291676,0.296725,0.319815,0.358653,...,-0.201487,0.283999,-0.196694,-0.200996,-0.191175,-0.190492,1.497924,1.221172,-0.104393,128209
14,-0.342119,-0.038115,-0.034881,-0.257335,-0.299326,-0.325458,-0.337929,-0.333823,-0.304903,-0.253256,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-0.657777,0.622895,-0.263850,65546
16,1.368040,0.200504,0.639991,-0.260748,-0.035209,-0.019994,0.051153,-0.355077,-1.217413,-1.425507,...,12.431747,3.891292,-0.196694,-0.200996,0.355947,-0.190492,-1.511717,-0.738170,-1.220596,145270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56996,1.135978,1.011198,1.012545,0.913389,0.892676,0.878843,0.875298,0.881222,0.898908,0.925871,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.702569,1.238249,0.939330,171231
57002,1.516952,0.283103,0.441500,0.039612,-0.288901,-0.332480,-0.256575,0.541151,0.088178,-0.490456,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.224753,-0.228696,0.605919,152212
57005,0.503633,1.317120,1.317918,1.254708,1.240199,1.229951,1.229008,1.235462,1.249873,1.269640,...,3.291514,2.243702,0.233102,0.259276,-0.191175,-0.190492,-0.082857,-1.659778,-0.524781,166991
57012,-0.755308,-1.185323,-0.923514,-1.062848,-0.893590,-0.883720,-0.914476,-1.095440,-1.340251,-1.109240,...,-0.201487,0.283999,1.836055,3.064335,1.450191,-0.190492,0.325247,-0.664737,-0.350827,46284


In [150]:
X_pluie_smote.shape

(28900, 149)

In [152]:
Y_test_smote = pd.DataFrame(Y_test)
Y_test_smote

,0
88974,2
6530,2
7976,2
83676,1
46503,2
...,...
166462,1
132111,2
46284,2
12082,2


In [153]:
X_pluie_smote

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,index_original
8,-0.656676,0.953073,0.954525,0.848538,0.826647,0.812132,0.808093,0.813916,0.832225,0.860555,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.531576,0.985504,0.214523,50882
10,0.449895,0.567611,0.569756,0.418476,0.388768,0.369736,0.362418,0.367573,0.390008,0.427406,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.066086,-1.056947,-0.539277,102468
13,1.002064,0.506426,0.508681,0.350213,0.319264,0.299514,0.291676,0.296725,0.319815,0.358653,...,-0.201487,0.283999,-0.196694,-0.200996,-0.191175,-0.190492,1.497924,1.221172,-0.104393,128209
14,-0.342119,-0.038115,-0.034881,-0.257335,-0.299326,-0.325458,-0.337929,-0.333823,-0.304903,-0.253256,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-0.657777,0.622895,-0.263850,65546
16,1.368040,0.200504,0.639991,-0.260748,-0.035209,-0.019994,0.051153,-0.355077,-1.217413,-1.425507,...,12.431747,3.891292,-0.196694,-0.200996,0.355947,-0.190492,-1.511717,-0.738170,-1.220596,145270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56996,1.135978,1.011198,1.012545,0.913389,0.892676,0.878843,0.875298,0.881222,0.898908,0.925871,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.702569,1.238249,0.939330,171231
57002,1.516952,0.283103,0.441500,0.039612,-0.288901,-0.332480,-0.256575,0.541151,0.088178,-0.490456,...,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.224753,-0.228696,0.605919,152212
57005,0.503633,1.317120,1.317918,1.254708,1.240199,1.229951,1.229008,1.235462,1.249873,1.269640,...,3.291514,2.243702,0.233102,0.259276,-0.191175,-0.190492,-0.082857,-1.659778,-0.524781,166991
57012,-0.755308,-1.185323,-0.923514,-1.062848,-0.893590,-0.883720,-0.914476,-1.095440,-1.340251,-1.109240,...,-0.201487,0.283999,1.836055,3.064335,1.450191,-0.190492,0.325247,-0.664737,-0.350827,46284


In [154]:
Y_pluie_smote = Class_smote.loc[X_pluie_smote['index_original']]
Y_pluie_smote.reset_index(inplace=True)
Y_pluie_smote.drop('index',axis=1,inplace=True)


Après avoir prédit les jours où il pleut, on utilise un modèle prédictif afin de déterminer le cumul de pluie par jour sur les jours où on prédit où il pleut

Sans SMOTE

In [ ]:
index_original = X_pluie['index_original']
X_pluie.drop('index_original',axis=1,inplace=True)

In [ ]:
X_pluie.reset_index(inplace=True)

In [ ]:
X_pluie.drop('index',axis=1,inplace=True)

In [ ]:
Y_pluie

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X_pluie, Y_pluie)
train_index, test_index = next(split.split(X_pluie, Y_pluie)) 

X_train,X_test = X_pluie.loc[train_index], X_pluie.loc[test_index]
Y_train, Y_test = Y_pluie.loc[train_index], Y_pluie.loc[test_index]


Avec SMOTE

In [157]:
index_original = X_pluie_smote['index_original']
X_pluie_smote.drop('index_original',axis=1,inplace=True)
X_pluie_smote.reset_index(inplace=True)
X_pluie_smote.drop('index',axis=1,inplace=True)



In [158]:
X_pluie_smote

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,-0.656676,0.953073,0.954525,0.848538,0.826647,0.812132,0.808093,0.813916,0.832225,0.860555,...,-0.194590,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.531576,0.985504,0.214523
1,0.449895,0.567611,0.569756,0.418476,0.388768,0.369736,0.362418,0.367573,0.390008,0.427406,...,-0.194590,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.066086,-1.056947,-0.539277
2,1.002064,0.506426,0.508681,0.350213,0.319264,0.299514,0.291676,0.296725,0.319815,0.358653,...,-0.194590,-0.201487,0.283999,-0.196694,-0.200996,-0.191175,-0.190492,1.497924,1.221172,-0.104393
3,-0.342119,-0.038115,-0.034881,-0.257335,-0.299326,-0.325458,-0.337929,-0.333823,-0.304903,-0.253256,...,-0.194590,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-0.657777,0.622895,-0.263850
4,1.368040,0.200504,0.639991,-0.260748,-0.035209,-0.019994,0.051153,-0.355077,-1.217413,-1.425507,...,0.311806,12.431747,3.891292,-0.196694,-0.200996,0.355947,-0.190492,-1.511717,-0.738170,-1.220596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28895,1.135978,1.011198,1.012545,0.913389,0.892676,0.878843,0.875298,0.881222,0.898908,0.925871,...,-0.194590,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,0.702569,1.238249,0.939330
28896,1.516952,0.283103,0.441500,0.039612,-0.288901,-0.332480,-0.256575,0.541151,0.088178,-0.490456,...,-0.194590,-0.201487,-0.196974,-0.196694,-0.200996,-0.191175,-0.190492,-1.224753,-0.228696,0.605919
28897,0.503633,1.317120,1.317918,1.254708,1.240199,1.229951,1.229008,1.235462,1.249873,1.269640,...,2.881492,3.291514,2.243702,0.233102,0.259276,-0.191175,-0.190492,-0.082857,-1.659778,-0.524781
28898,-0.755308,-1.185323,-0.923514,-1.062848,-0.893590,-0.883720,-0.914476,-1.095440,-1.340251,-1.109240,...,-0.194590,-0.201487,0.283999,1.836055,3.064335,1.450191,-0.190492,0.325247,-0.664737,-0.350827


In [159]:
Y_pluie_smote

,0
0,2
1,2
2,2
3,2
4,2
...,...
28895,1
28896,2
28897,1
28898,2


In [160]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3,random_state = 4)
split.get_n_splits(X_pluie_smote, Y_pluie_smote)
train_index, test_index = next(split.split(X_pluie_smote, Y_pluie_smote)) 

X_train,X_test = X_pluie_smote.loc[train_index], X_pluie_smote.loc[test_index]
Y_train, Y_test = Y_pluie_smote.loc[train_index], Y_pluie_smote.loc[test_index]

Utilisons une régression linéaire


In [161]:
from sklearn import linear_model, metrics

reg = linear_model.LinearRegression()

reg.fit(X_train,Y_train)



LinearRegression()

In [162]:
predictions = reg.predict(X_test)
predictions

array([[1.91750439],
       [1.74232049],
       [1.78261713],
       ...,
       [1.76402136],
       [1.7617844 ],
       [1.87582603]])

In [163]:
np.mean(np.abs(predictions-Y_test)/(Y_test+1))*100

0    14.313676
dtype: float64

In [164]:
metrics.mean_absolute_error(predictions,Y_test)

0.34463857919174695

Random Forest Regression

In [165]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(random_state=4)
regressor.fit(X_train, Y_train)
predictions = regressor.predict(X_test)

In [166]:
new_predictions = [[i] for i in predictions]
new_predictions = np.asarray(new_predictions)

In [167]:
np.mean(np.abs(new_predictions-Y_test)/(Y_test+1))*100

0    13.215475
dtype: float64

In [168]:
metrics.mean_absolute_error(predictions,Y_test)

0.32116493656286044

Quantile regression

In [169]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_pinball_loss, mean_squared_error


all_models = {}
common_params = dict(
    learning_rate=0.05,
    n_estimators=100,
    max_depth=2,
    min_samples_leaf=9,
    min_samples_split=9,
)
for alpha in [0.05]:
    gbr = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
    all_models["model_1"] = gbr.fit(X_train, Y_train)

In [170]:
predictions = all_models['model_1'].predict(X_test)


In [171]:
quantile_regression_pred = [[i] for i in predictions]
quantile_regression_pred = np.asarray(quantile_regression_pred)

In [172]:
np.mean(np.abs(quantile_regression_pred-Y_test)/(Y_test+1))*100

0    24.176642
dtype: float64

Prédictions sur données Test

In [173]:
# Lecture des données de l'ensemble test

path = 'defi-ia-2022/Test/Test/X_station_test.csv'


with open(path, encoding="utf8", errors='ignore') as f:
  contents = f.read()
Xtest = pd.read_csv(path)
display(Xtest)
#Xtest['number_sta']=Xtest['number_sta'].astype('category')

# Tri par station puis par datea
#Xtest = Xtest.sort_values(['number_sta','date'])
#Xtest

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


In [174]:
split_Id = Xtest['Id'].str.split(pat="_", expand = True)
split_Id = split_Id.rename(columns={0: "number_sta_2", 1: "day", 2: "hour"})
Xtest['number_sta_2'] = split_Id['number_sta_2']
Xtest['day'] = split_Id["day"]
Xtest['hour'] = split_Id["hour"]
Xtest['Id'] = split_Id['number_sta_2'] + "_" + split_Id['day']
Xtest = Xtest.drop("number_sta_2",axis=1)
Xtest.fillna(method='bfill',inplace=True)
display(Xtest)

,dd,hu,td,t,ff,precip,month,Id,day,hour
0,221.0,89.3,275.19,278.35,4.90,0.0,12,14047002_277,277,4
1,221.0,89.3,275.19,278.40,4.90,0.0,12,14047002_277,277,5
2,221.0,89.3,275.19,279.01,4.90,0.0,12,14047002_277,277,6
3,221.0,89.3,275.19,279.66,4.90,0.0,12,14047002_277,277,7
4,221.0,89.3,275.19,279.99,4.90,0.0,12,14047002_277,277,8
...,...,...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176,176,19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176,176,20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176,176,21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176,176,22


In [175]:
Xtest['ff_idx'] = 'ff_' + Xtest["hour"].astype(str)
Xtest['t_idx'] = 't_' + Xtest["hour"].astype(str)
Xtest['td_idx'] = 'td_' + Xtest["hour"].astype(str)
Xtest['hu_idx'] = 'hu_' + Xtest["hour"].astype(str)
Xtest['dd_idx'] = 'dd_' + Xtest["hour"].astype(str)
Xtest['precip_idx'] = 'precip_' + Xtest["hour"].astype(str)

In [176]:
ff = Xtest.pivot(index='Id',columns='ff_idx',values='ff')
t = Xtest.pivot(index='Id',columns='t_idx',values='t')
td = Xtest.pivot(index='Id',columns='td_idx',values='td')
hu = Xtest.pivot(index='Id',columns='hu_idx',values='hu')
dd = Xtest.pivot(index='Id',columns='dd_idx',values='dd')
precip = Xtest.pivot(index='Id',columns='precip_idx',values='precip')

In [177]:
X_reshape = pd.concat([ff ,t,td,hu,dd,precip],axis=1)

In [178]:
X_reshape.reset_index(inplace=True)
X_reshape

,Id,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_21,precip_22,precip_23,precip_3,precip_4,precip_5,precip_6,precip_7,precip_8,precip_9
0,14047002_100,5.38,5.38,5.38,5.38,5.38,5.38,5.38,5.380000,5.38,...,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,14047002_223,4.54,4.54,4.54,4.54,4.54,4.54,4.54,4.540000,4.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14047002_236,3.57,3.57,3.57,3.57,3.57,3.57,3.57,3.570000,3.57,...,0.0,0.0,0.6,0.2,0.6,0.0,0.0,0.8,0.0,0.0
3,14047002_256,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.000000,2.00,...,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14047002_277,NaN,NaN,4.90,4.90,4.90,4.90,4.90,4.900000,4.90,...,2.4,2.0,0.6,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96050,95690001_95,0.13,0.30,2.37,1.82,2.20,2.44,2.32,2.140000,2.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96051,95690001_96,3.15,3.92,6.27,5.49,6.19,7.46,6.06,4.710000,5.00,...,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96052,95690001_97,1.41,1.76,6.33,8.23,7.77,6.50,4.28,3.520000,2.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96053,95690001_98,6.90,7.09,8.85,8.58,7.42,7.62,9.56,8.250000,7.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


In [179]:
X = X_reshape

X['day'] = X['Id'].str.rpartition("_")[2]
X['day'] = X['day'].astype('int')
X['day'] = X['day'] % 365

In [180]:
X['number_sta'] = X['Id'].str.rpartition('_')[0]
X['number_sta'] = X['number_sta'].astype('int64')

In [181]:
X['season'] =pd.cut(X['day'],[0,60,151,243,334], labels=['Hiver','Printemps','Ete', 'Automne'],ordered=False)
X['season'] = X['season'].fillna('Hiver')
X['season']

0        Printemps
1              Ete
2              Ete
3          Automne
4          Automne
           ...    
96050    Printemps
96051    Printemps
96052    Printemps
96053    Printemps
96054    Printemps
Name: season, Length: 96055, dtype: category
Categories (4, object): ['Hiver', 'Printemps', 'Ete', 'Automne']

In [182]:
X['month'] = X['day'] / 30
X['month'] = X['month'].astype('int')

In [183]:
X.reset_index(inplace=True)
X.drop('index',axis=1,inplace=True)

In [184]:
X['month'] = X['month'].astype('category')
X['season'] = X['season'].astype('category')
X['day'] = X['day'].astype('category')

In [185]:
X =pd.merge(X,coords,how='left',on='number_sta')
X['number_sta'] = X['number_sta'].astype('category')
X.fillna(method = 'bfill',inplace = True)

Sans SMOTE

In [ ]:

X_test = transformer.fit_transform(X)
predictions_test = random_forest.predict(X_test)

In [ ]:
predictions_test

In [ ]:
index_pluie =np.where(predictions_test==2)

In [ ]:
X_pluie = X.loc[index_pluie]
X_pluie

In [ ]:
X_pluie_test = transformer.fit_transform(X_pluie)

In [ ]:
predictions_test_regressor = regressor.predict(X_pluie_test)

In [ ]:
predictions_test_regressor_quant = all_models['model_1'].predict(X_pluie_test)

In [ ]:
array_test = pd.DataFrame(predictions_test)
array_test[0] = np.where(array_test[0]==1,0,2)
array_test['Id'] = X['Id']
display(array_test)

In [ ]:
array_test.set_axis(['pred', 'Id'], axis=1, inplace=True)

In [ ]:
array_test

In [ ]:
array_test.pred.value_counts()

In [ ]:
tab_reg = {'pred' : predictions_test_regressor ,'Id' : X_pluie['Id']}
tab_reg = pd.DataFrame(tab_reg)
tab_reg

In [ ]:
tab_reg_quant = {'pred_quant' : predictions_test_regressor_quant ,'Id' : X_pluie['Id']}
tab_reg_quant = pd.DataFrame(tab_reg_quant)
tab_reg_quant

In [ ]:
pred = pd.merge(array_test,tab_reg,on ='Id',how='left')
pred

In [ ]:
pred['pred_y'] = pred.pred_y.fillna(0)
pred.drop('pred_x',axis=1,inplace=True)
pred.rename(columns = {'pred_y' : 'pred'},inplace=True)
pred

In [ ]:
pred = pd.merge(pred,tab_reg_quant,on ='Id',how='left')
pred

In [ ]:
pred['pred_quant'] = pred.pred_quant.fillna(0)
pred

In [ ]:
fname = './defi-ia-2022/Test/Test/Baselines/Baseline_observation_test.csv'
Baseline_observation_test = pd.read_csv(fname)
Baseline_observation_test = Baseline_observation_test.sort_values(by=["Id"])
display(Baseline_observation_test)

In [ ]:
pred = pd.merge(Baseline_observation_test,pred,on ='Id',how='left')
pred

In [ ]:
pred = pred.drop(["Prediction","pred"],axis=1)
pred

In [ ]:
pred.rename(columns = {'pred_quant' : 'Prediction'},inplace=True)
pred['Prediction']=pred['Prediction']+1
pred

In [ ]:
# Enregistrement prédictions

output_file = "prediction_test.csv"
pred.to_csv('./predictions/' + output_file,index=False)

Avec SMOTE

In [212]:
X

,Id,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_7,precip_8,precip_9,day,number_sta,season,month,lat,lon,height_sta
0,14047002_100,5.38,5.38,5.38,5.38,5.38,5.38,5.38,5.380000,5.38,...,0.0,0.0,1.0,100,14047002,Printemps,3,49.275,-0.712,60.0
1,14047002_223,4.54,4.54,4.54,4.54,4.54,4.54,4.54,4.540000,4.54,...,0.0,0.0,0.0,223,14047002,Ete,7,49.275,-0.712,60.0
2,14047002_236,3.57,3.57,3.57,3.57,3.57,3.57,3.57,3.570000,3.57,...,0.8,0.0,0.0,236,14047002,Ete,7,49.275,-0.712,60.0
3,14047002_256,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.000000,2.00,...,0.0,0.0,0.0,256,14047002,Automne,8,49.275,-0.712,60.0
4,14047002_277,3.39,3.39,4.90,4.90,4.90,4.90,4.90,4.900000,4.90,...,0.0,0.0,0.0,277,14047002,Automne,9,49.275,-0.712,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96050,95690001_95,0.13,0.30,2.37,1.82,2.20,2.44,2.32,2.140000,2.21,...,0.0,0.0,0.0,95,95690001,Printemps,3,49.108,1.831,126.0
96051,95690001_96,3.15,3.92,6.27,5.49,6.19,7.46,6.06,4.710000,5.00,...,0.0,0.0,0.0,96,95690001,Printemps,3,49.108,1.831,126.0
96052,95690001_97,1.41,1.76,6.33,8.23,7.77,6.50,4.28,3.520000,2.42,...,0.0,0.0,0.0,97,95690001,Printemps,3,49.108,1.831,126.0
96053,95690001_98,6.90,7.09,8.85,8.58,7.42,7.62,9.56,8.250000,7.16,...,0.2,0.0,0.0,98,95690001,Printemps,3,49.108,1.831,126.0


In [208]:
X_test_smote = transformer.fit_transform(X)

predictions_test_smote = extree.predict(X_test_smote)

In [213]:
index_pluie_smote =np.where(predictions_test_smote==2)

In [214]:
X_pluie_smote = X.loc[index_pluie_smote]
X_pluie_smote

,Id,ff_0,ff_1,ff_10,ff_11,ff_12,ff_13,ff_14,ff_15,ff_16,...,precip_7,precip_8,precip_9,day,number_sta,season,month,lat,lon,height_sta
0,14047002_100,5.38,5.38,5.38,5.38,5.38,5.38,5.38,5.380000,5.38,...,0.0,0.0,1.0,100,14047002,Printemps,3,49.275,-0.712,60.0
1,14047002_223,4.54,4.54,4.54,4.54,4.54,4.54,4.54,4.540000,4.54,...,0.0,0.0,0.0,223,14047002,Ete,7,49.275,-0.712,60.0
2,14047002_236,3.57,3.57,3.57,3.57,3.57,3.57,3.57,3.570000,3.57,...,0.8,0.0,0.0,236,14047002,Ete,7,49.275,-0.712,60.0
3,14047002_256,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.000000,2.00,...,0.0,0.0,0.0,256,14047002,Automne,8,49.275,-0.712,60.0
4,14047002_277,3.39,3.39,4.90,4.90,4.90,4.90,4.90,4.900000,4.90,...,0.0,0.0,0.0,277,14047002,Automne,9,49.275,-0.712,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96047,95690001_92,1.57,2.33,0.93,1.41,1.46,1.13,2.32,1.090000,2.19,...,0.0,0.0,0.0,92,95690001,Printemps,3,49.108,1.831,126.0
96048,95690001_93,6.55,6.17,7.73,9.19,8.24,7.07,6.76,6.180000,7.52,...,0.0,0.0,0.0,93,95690001,Printemps,3,49.108,1.831,126.0
96051,95690001_96,3.15,3.92,6.27,5.49,6.19,7.46,6.06,4.710000,5.00,...,0.0,0.0,0.0,96,95690001,Printemps,3,49.108,1.831,126.0
96053,95690001_98,6.90,7.09,8.85,8.58,7.42,7.62,9.56,8.250000,7.16,...,0.2,0.0,0.0,98,95690001,Printemps,3,49.108,1.831,126.0


In [215]:
X_pluie_test_smote = transformer.fit_transform(X_pluie_smote)

In [216]:
predictions_test_regressor_smote = regressor.predict(X_pluie_test_smote)

In [228]:
array_test = pd.DataFrame(predictions_test_smote)
array_test[0] = np.where(array_test[0]==1,0,2)
array_test['Id'] = X['Id']
display(array_test)

,0,Id
0,2,14047002_100
1,2,14047002_223
2,2,14047002_236
3,2,14047002_256
4,2,14047002_277
...,...,...
96050,2,95690001_95
96051,2,95690001_96
96052,2,95690001_97
96053,2,95690001_98


In [229]:
array_test.set_axis(['pred', 'Id'], axis=1, inplace=True)

In [230]:
tab_reg = {'pred' : predictions_test_regressor_smote ,'Id' : X_pluie_smote['Id']}
tab_reg = pd.DataFrame(tab_reg)
tab_reg

,pred,Id
0,1.88,14047002_100
1,1.50,14047002_223
2,1.73,14047002_236
3,1.02,14047002_256
4,1.75,14047002_277
...,...,...
96047,1.70,95690001_92
96048,1.62,95690001_93
96051,1.74,95690001_96
96053,1.64,95690001_98


In [231]:
pred = pd.merge(array_test,tab_reg,on ='Id',how='left')
pred

,pred_x,Id,pred_y
0,2,14047002_100,1.88
1,2,14047002_223,1.50
2,2,14047002_236,1.73
3,2,14047002_256,1.02
4,2,14047002_277,1.75
...,...,...,...
96050,2,95690001_95,NaN
96051,2,95690001_96,1.74
96052,2,95690001_97,NaN
96053,2,95690001_98,1.64


In [232]:
pred['pred_y'] = pred.pred_y.fillna(0)
pred.drop('pred_x',axis=1,inplace=True)
pred.rename(columns = {'pred_y' : 'pred'},inplace=True)
pred

,Id,pred
0,14047002_100,1.88
1,14047002_223,1.50
2,14047002_236,1.73
3,14047002_256,1.02
4,14047002_277,1.75
...,...,...
96050,95690001_95,0.00
96051,95690001_96,1.74
96052,95690001_97,0.00
96053,95690001_98,1.64


In [233]:
fname = './defi-ia-2022/Test/Test/Baselines/Baseline_observation_test.csv'
Baseline_observation_test = pd.read_csv(fname)
Baseline_observation_test = Baseline_observation_test.sort_values(by=["Id"])
display(Baseline_observation_test)

,Id,Prediction
82989,14047002_100,9.9
84063,14047002_223,0.8
83793,14047002_236,7.0
84334,14047002_256,4.0
83254,14047002_281,0.0
...,...,...
21605,95690001_95,0.0
43472,95690001_96,1.8
45972,95690001_97,0.0
3055,95690001_98,8.0


In [243]:
pred_submission = pd.merge(Baseline_observation_test,pred,on ='Id',how='left')
pred_submission

,Id,Prediction,pred
0,14047002_100,9.9,1.88
1,14047002_223,0.8,1.50
2,14047002_236,7.0,1.73
3,14047002_256,4.0,1.02
4,14047002_281,0.0,1.74
...,...,...,...
85135,95690001_95,0.0,0.00
85136,95690001_96,1.8,1.74
85137,95690001_97,0.0,0.00
85138,95690001_98,8.0,1.64


In [244]:
pred_submission = pred_submission.drop("Prediction",axis=1)
pred_submission.rename(columns = {'pred' : 'Prediction'},inplace=True)


On ajoute +1 à nos prédictions car le défi utilise l'erreur MAPE et cela évite la division par 0

In [246]:
pred_submission['Prediction']=pred_submission['Prediction']+1

In [247]:
pred_submission

,Id,Prediction
0,14047002_100,2.88
1,14047002_223,2.50
2,14047002_236,2.73
3,14047002_256,2.02
4,14047002_281,2.74
...,...,...
85135,95690001_95,1.00
85136,95690001_96,2.74
85137,95690001_97,1.00
85138,95690001_98,2.64


In [248]:
# Enregistrement prédictions

output_file = "prediction_test.csv"
pred_submission.to_csv('./predictions/' + output_file,index=False)

Utilisons les données 2D

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import cfgrib

In [ ]:
from netCDF4 import Dataset
import h5py

In [ ]:
model = 'arome'  #weather model (arome or arpege)
param = "tp"   #parameter name in the file (cf cells below to know the parameter names -> exploration of metadata)
file_date = dt.datetime(2017, 2, 14) # Day example 

In [ ]:
fname = "defi-ia-2022/Train/Train/X_forecast/2D_%s_%s.nc" %(model,file_date.strftime('%Y%m%d'))
data = xr.load_dataset(fname)
#data

In [ ]:
from scipy.io import netcdf
import netCDF4
nc = netCDF4.Dataset(fname)
nc.variables.keys()

In [ ]:
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']
latitude = nc.variables['latitude']

In [ ]:
nc.variables['Id']